## Business Solution:
This project aims to develop a product that creates brochures for a company such as NVIDIA. The brochures will be designed for prospective clients, investors, and potential recruits and translated into several languages, including Arabic, Hebrew, and Japanese, using large language models and OpenAI technologies.

In [1]:
#imports libraries

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key=os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? ")

MODEL='gpt-4o-mini'
openai=OpenAI()


API key looks good so far


In [4]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [12]:
link_system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt+="You should respond in JSON as in this example:"
link_system_prompt+="""
{
    "links":[
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [13]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links":[
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [14]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [16]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)

In [19]:
def get_all_details(url):
    result="Landing page:\n"
    result+=Website(url).get_contents()
    links=get_links(url)
    print("Found links:",links)
    for link in links["links"]:
        result+=f"\n\n{link['type']}\n"
        result+=Website(link["url"]).get_contents()
    return result

In [36]:
# print(get_all_details("https://www.nvidia.com"))

In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [35]:
# get_brochure_user_prompt("NVIDIA", "https://www.nvidia.com")

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("NVIDIA", "https://www.nvidia.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nvidia.com/en-au/about-nvidia/'}, {'type': 'careers page', 'url': 'https://www.nvidia.com/en-us/about-nvidia/careers/'}, {'type': 'company page', 'url': 'https://www.nvidia.com/en-us/about-nvidia/'}, {'type': 'executive insights', 'url': 'https://www.nvidia.com/en-au/executive-insights/'}, {'type': 'training page', 'url': 'https://www.nvidia.com/en-au/training/'}]}


# NVIDIA Company Brochure

## About Us
As a world leader in artificial intelligence computing, **NVIDIA** is dedicated to revolutionizing industries through its innovative hardware and software solutions. Established with a vision to facilitate breakthroughs in AI and graphics technology, NVIDIA continues to empower enterprises and creators globally, delivering performance, efficiency, and transformative capabilities across various fields.

---

## Products & Solutions
NVIDIA's extensive portfolio includes:

- **Graphics Processing Units (GPUs)** for gaming, professional visualization, and data centers.
- **AI Solutions** leveraging frameworks such as Triton for AI inference and NeMo for conversational AI.
- **Supercomputing Technologies**, including the DGX platform for AI training and inference.
- **Edge Computing** innovations like the Jetson platform.
- **Omniverse**, a platform designed for 3D content creation and simulation.

Our solutions cater to a wide array of industries, including gaming, healthcare, automotive, financial services, and more.

---

## Customer Base
NVIDIA serves a diverse clientele, from individual gamers and content creators to large enterprises and research institutions. Our technologies are essential across various sectors, enabling advancements in:

- **Game Development**: Graphic-intensive games powered by GeForce RTX technology.
- **Healthcare**: Innovations in medical imaging and genomics powered by AI.
- **Automotive**: Intelligent systems for self-driving vehicles and in-vehicle computing.

---

## Company Culture
At NVIDIA, we foster a **collaborative and inclusive culture** that values creativity, innovation, and diversity. Our teams are passionate about technology and are committed to pushing the boundaries of AI and computing. We encourage continuous learning and support professional development, enabling our employees to thrive in a dynamic and rapidly evolving environment.

---

## Careers at NVIDIA
NVIDIA offers a wide range of career opportunities for associates at different levels:

- **Engineering & Development**: Positions in software engineering, hardware design, and AI research.
- **Sales & Marketing**: Roles focusing on customer engagement, product marketing, and partnerships.
- **Operations**: Careers ranging from project management to supply chain solutions.

Join us in our mission to unleash the transformative power of technology. **Explore our job openings** and find your place at NVIDIA today!

---

## Connect with Us
For more information about our products, solutions, and career opportunities, please visit our website: [NVIDIA](https://www.nvidia.com) 

**NVIDIA - Big Ideas, Powerful Innovations.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream= openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response=""
    display_handle=display(Markdown(""),display_id=True)
    for chunk in stream:
        response +=chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown","")
        update_display(Markdown(response),display_id=display_handle.display_id)

In [27]:

stream_brochure("NVIDIA", "https://www.nvidia.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nvidia.com/en-au/about-nvidia/'}, {'type': 'careers page', 'url': 'https://www.nvidia.com/en-us/about-nvidia/careers/'}, {'type': 'company page', 'url': 'https://www.nvidia.com/en-us/about-nvidia/'}, {'type': 'investor relations', 'url': 'https://investor.nvidia.com/home/default.aspx'}, {'type': 'executive insights', 'url': 'https://www.nvidia.com/en-au/executive-insights/'}, {'type': 'training page', 'url': 'https://www.nvidia.com/en-au/training/'}]}


# NVIDIA Company Brochure

---

## **About Us**
NVIDIA is a global leader in artificial intelligence (AI) computing. We specialize in bringing AI advancements to multiple industries, including gaming, design, healthcare, education, and transportation. Originally known for pioneering visual computing, our capabilities have expanded to encompass powerful AI platforms that support both enterprise and individual needs.

---

## **Our Products & Solutions**
- **Hardware**: Including GeForce graphics cards, RTX AI PCs, and high-performance servers for data centers.
- **Software**: Innovative AI applications such as Triton for AI inference, RAPIDS for data analytics, and the Omniverse for interactive 3D design.
- **Industries Served**:
  - Architecture, Engineering, and Construction
  - Consumer Internet
  - Cybersecurity
  - Financial Services
  - Healthcare and Life Sciences
  - Game Development

---

## **Customer Base**
Our diverse customer base ranges from developers and IT professionals to creative designers and researchers. NVIDIA solutions empower them to create, analyze, and optimize their workflows using cutting-edge AI technology.

---

## **Company Culture**
At NVIDIA, we foster a culture of innovation and collaboration. We encourage our employees to push the boundaries of technology and creativity. Our team comprises passionate individuals who are dedicated to transforming industries through AI. We embrace diversity and inclusion, recognizing that diverse thoughts enhance problem-solving and innovation.

---

## **Careers at NVIDIA**
Joining NVIDIA offers enthusiasts and experts the opportunity to work on groundbreaking projects in AI and computing. We seek talented individuals who are eager to make an impact and contribute to advancements in technology. Our employees benefit from:
- **Professional Development**: Training and mentorship opportunities to advance their careers.
- **Inclusive Environment**: A workplace that embraces diverse perspectives and backgrounds.
- **Competitive Benefits**: Comprehensive packages that ensure the well-being of our employees.

---

## **Conclusion**
NVIDIA is at the forefront of AI innovation, ready to empower the next generation of technology solutions. Whether you're a potential customer looking for cutting-edge products, an investor interested in our growth trajectory, or a recruit wanting to join our passionate team, NVIDIA welcomes you to be a part of our exciting journey.

For more information, visit us at [NVIDIA](https://www.nvidia.com) or connect with us through our careers page to explore opportunities.

--- 

### *Transforming the Future with Artificial Intelligence*

# Convert the brochure to Arabic language


In [28]:
def create_brochure_language(company_name, url, language):
    language_prompt = f"You are a professional translator and writer specializing in creating and translating brochures. Convert the brochure to {language} while maintaining its original tone, format, and purpose."
    user_language_prompt = f"Generate a brochure for the company '{company_name}' available at the URL: {url}, and translate it into {language}."
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": language_prompt},
            {"role": "user", "content": user_language_prompt}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))


In [29]:
create_brochure_language("NVIDIA", "https://www.nvidia.com","Arabic")

I'm unable to access external URLs directly, but I can help you create a sample brochure for NVIDIA and then translate it into Arabic. Below is an example brochure text you might find on NVIDIA's website, followed by its Arabic translation.

---

**NVIDIA Brochure**

**Unleashing the Power of AI and Graphics**

**About Us**
NVIDIA is a leading innovator in visual computing technologies and artificial intelligence. Our cutting-edge graphics processing units (GPUs) power everything from gaming and entertainment to deep learning and data science.

**Our Products**
1. **Gaming:** Experience immersive gameplay with NVIDIA GeForce GPUs, designed to deliver superior graphics and frame rates.
2. **AI & Deep Learning:** Leverage GPU-accelerated computing with NVIDIA Tesla and A100 GPUs for powerful AI applications.
3. **Data Centers:** Optimize your data infrastructure with NVIDIA DGX systems and networking solutions to enhance performance and efficiency.

**Why Choose NVIDIA?**
- **Innovative Technology:** Stay ahead with our state-of-the-art products and solutions.
- **Community & Support:** Join a thriving community and access extensive support for developers and gamers alike.
- **Sustainability:** Committed to sustainability, we ensure our technologies are energy-efficient and environmentally friendly.

**Join Us**
Discover how NVIDIA can transform your experience. Visit our website at [www.nvidia.com](http://www.nvidia.com) for more information and to explore our latest developments.

---

**كتيب إنفيديا**

**تحرير قوة الذكاء الاصطناعي والرسومات**

**من نحن**
تعتبر إنفيديا رائدة في الابتكار في تكنولوجيا الحوسبة البصرية والذكاء الاصطناعي. تقوم وحدات معالجة الرسومات المتطورة لدينا بتشغيل كل شيء من الألعاب والترفيه إلى التعلم العميق وعلوم البيانات.

**منتجاتنا**
1. **الألعاب:** استمتع بتجربة لعب غامرة مع وحدات معالجة الرسوميات NVIDIA GeForce، المصممة لتقديم رسومات ومعدلات إطار متفوقة.
2. **الذكاء الاصطناعي والتعلم العميق:** استغل الحوسبة المعززة لوحدات معالجة الرسوم NVIDIA Tesla وA100 لتطبيقات الذكاء الاصطناعي القوية.
3. **مراكز البيانات:** قم بتحسين بنيتك التحتية للبيانات باستخدام أنظمة NVIDIA DGX وحلول الشبكات لتعزيز الأداء والكفاءة.

**لماذا تختار إنفيديا؟**
- **تكنولوجيا مبتكرة:** ابق في الطليعة مع منتجاتنا وحلولنا المتطورة.
- **مجتمع ودعم:** انضم إلى مجتمع مزدهر واحصل على دعم واسع للمطورين واللاعبين على حد سواء.
- **الاستدامة:** ملتزمون بالاستدامة، نضمن أن تكون تقنياتنا موفرة للطاقة وصديقة للبيئة.

**انضم إلينا**
اكتشف كيف يمكن لإنفيديا تحويل تجربتك. قم بزيارة موقعنا على الإنترنت على [www.nvidia.com](http://www.nvidia.com) لمزيد من المعلومات واستكشاف أحدث تطوراتنا.

--- 

Feel free to modify any section according to your requirements!

# Convert the brochure to the Hebrew language

In [30]:
create_brochure_language("NVIDIA", "https://www.nvidia.com","Hebrew")

I’m unable to directly access external content such as the NVIDIA website. However, I can help you create a sample brochure for NVIDIA based on general information and then translate it into Hebrew. Please find the sample brochure below:

---

### **NVIDIA: Pioneering the Future of Computing**

**About Us:**
NVIDIA is a global leader in accelerated computing. Our technology is transforming industries and driving progress in artificial intelligence, gaming, data centers, and automotive.

**Our Technology:**
- **Gaming:** Experience immersive gaming with the latest RTX graphics cards that deliver real-time ray tracing and AI-enhanced visuals.
- **Data Science:** Leverage powerful GPUs for faster data processing and AI modeling, helping businesses make informed decisions.
- **AI Innovation:** NVIDIA is at the forefront of AI research, providing tools and frameworks that empower developers worldwide.

**Our Products:**
1. **GeForce RTX Series:** Elevate your gaming experience with cutting-edge graphics performance.
2. **NVIDIA DGX Systems:** Designed for AI research, these systems provide unparalleled computing power.
3. **NVIDIA Jetson:** An edge computing platform to bring AI capabilities to devices.

**Why Choose NVIDIA?**
- **Innovation:** We are committed to pushing the boundaries of technology.
- **Community:** Join a vibrant community of developers, gamers, and researchers.
- **Support:** Our dedicated support teams are here to help you maximize your experience.

**Get Involved:**
Visit our website to learn more about our products, download drivers, and join our community.

**Contact Us:**
For inquiries, please reach out through our website or follow us on social media.

---

### **NVIDIA: מובילים את העתיד של המחשוב**

**עלינו:**
NVIDIA היא המובילה הגלובלית בתחום המחשוב המואץ. הטכנולוגיה שלנו משנה תעשיות ומניעה התקדמות בתחום הבינה המלאכותית, משחקים, מרכזי נתונים ורכב.

**הטכנולוגיה שלנו:**
- **משחקים:** חווה חוויית משחק מרהיבה עם כרטיסי מסך RTX החדשים שמספקים ריכוך קרני אור בזמן אמת ומראות משופרים באינטליגנציה מלאכותית.
- **מדעי הנתונים:** נצל מעבדים גרפיים רבי עוצמה לעיבוד נתונים מהיר ולמודלים של בינה מלאכותית, המסייעים לעסקים לקבל החלטות מושכלות.
- **חדשנות בינה מלאכותית:** NVIDIA נמצאת בחזית מחקר הבינה המלאכותית, ומספקת כלים ומסגרות המעצימות מפתחים ברחבי העולם.

**המוצרים שלנו:**
1. **סדרת GeForce RTX:** שדרג את חוויית המשחק שלך עם ביצועי גרפיקה מתקדמים.
2. **מערכות NVIDIA DGX:** מיועדות למחקר בינה מלאכותית, מערכות אלו מספקות כוח חישוב חסר תקדים.
3. **NVIDIA Jetson:** פלטפורמת חישוב קצה להבאת יכולות בינה מלאכותית למכשירים.

**למה לבחור ב-NVIDIA?**
- **חדשנות:** אנו מחויבים לדחוף את גבולות הטכנולוגיה.
- **קהילה:** הצטרף לקהילה התוססת של מפתחים, גיימרים וחוקרים.
- **תמיכה:** צוותי התמיכה המיועדים שלנו כאן כדי לעזור לך למקסם את החוויה שלך.

**התקשר אלינו:**
בקר באתר האינטרנט שלנו כדי ללמוד עוד על המוצרים שלנו, להוריד מנהלי התקן ולהצטרף לקהילה שלנו.

---

Feel free to modify any sections to better match specific content or features you want to highlight!

# Convert the brochure to the Japanese language

In [31]:
create_brochure_language("NVIDIA", "https://www.nvidia.com","Japanese")

Sure, I can help create a sample brochure for NVIDIA and then translate it into Japanese. Below is a fictional brochure based on common themes associated with the company. Please keep in mind, this is an original creation and will not be directly from the NVIDIA website.

---

**NVIDIA Brochure**

**Unleashing the Power of AI and Graphics**

Welcome to NVIDIA - where innovation meets performance. Our cutting-edge technology is revolutionizing industries by providing powerful tools for creators, gamers, and businesses worldwide.

**Our Products:**

1. **GeForce Gaming Graphics Cards**
   - Experience unparalleled gaming performance with our latest GPUs, designed to deliver stunning visuals and smooth gameplay.
   - Explore our range: from the entry-level GeForce GTX to the flagship GeForce RTX.

2. **NVIDIA AI Solutions**
   - Transform your business with AI-driven applications.
   - Our frameworks, such as CUDA and TensorRT, empower developers to build smarter, more efficient systems.

3. **NVIDIA Studio**
   - Designed for creators, NVIDIA Studio provides the software and hardware needed for graphic design, video editing, and 3D rendering.
   - Powered by RTX technology for real-time ray tracing and enhanced workflows.

4. **Data Center Solutions**
   - Harness the power of AI and deep learning with NVIDIA’s data center GPUs.
   - Optimize performance for computing tasks and analytics across various industries.

**Why Choose NVIDIA?**

- **Innovation**: Pioneering advancements in AI, gaming, and professional graphics.
- **Performance**: High-quality products backed by trusted performance metrics.
- **Community**: Join a vibrant community of gamers and developers contributing to the future of technology.

**Get in Touch**

For more information about our products and services, visit our website: [www.nvidia.com](http://www.nvidia.com)

---

**NVIDIA ブロシュア**

**AI とグラフィックスの力を解き放つ**

NVIDIA へようこそ - 革新とパフォーマンスが出会う場所。当社の最先端技術は、創作者、ゲーマー、ビジネスのための強力なツールを提供することで、世界中の業界に革命をもたらしています。

**製品一覧:**

1. **GeForce ゲーミンググラフィックスカード**
   - 最新の GPU で比類のないゲーミングパフォーマンスを体験してください。驚くべきビジュアルとスムーズなゲームプレイを実現するために設計されています。
   - エントリーレベルの GeForce GTX からフラッグシップの GeForce RTX までの幅広いラインアップをご覧ください。

2. **NVIDIA AI ソリューション**
   - AI 主導のアプリケーションでビジネスを変革しましょう。
   - CUDA や TensorRT などのフレームワークにより、開発者はよりスマートで効率的なシステムを構築できます。

3. **NVIDIA Studio**
   - 創作者向けに設計された NVIDIA Studio は、グラフィックデザイン、ビデオ編集、3D レンダリングに必要なソフトウェアとハードウェアを提供します。
   - リアルタイムレイトレーシングと強化されたワークフローのために RTX 技術により強化されています。

4. **データセンターソリューション**
   - NVIDIA のデータセンター GPU で AI と深層学習の力を活用します。
   - 様々な業界向けのコンピューティングタスクや分析のためにパフォーマンスを最適化します。

**NVIDIA を選ぶ理由**

- **革新**: AI、ゲーミング、プロフェッショナルグラフィックスにおける先駆的な進展。
- **パフォーマンス**: 信頼されたパフォーマンスメトリクスに裏付けられた高品質な製品。
- **コミュニティ**: テクノロジーの未来に貢献するゲーマーや開発者の活気あるコミュニティに参加しましょう。

**お問い合わせ**

製品やサービスに関する詳細は、当社のウェブサイトをご覧ください: [www.nvidia.com](http://www.nvidia.com)

---

This translation maintains the original tone, format, and purpose of the fictional NVIDIA brochure.